In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


def calculate_angle(p1, p2, p3):
    v1 = p1 - p2
    v2 = p3 - p2
    angle_radians = np.arccos(
        np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    )
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees


def get_pose_landmarks(cap):
    with mp_pose.Pose(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as pose:
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                break

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            results = pose.process(image_rgb)
            landmarks = results.pose_landmarks.landmark

            if landmarks:
                shoulder_index = 11
                elbow_index = 13
                wrist_index = 15

                shoulder = np.array(
                    [landmarks[shoulder_index].x, landmarks[shoulder_index].y]
                )
                elbow = np.array([landmarks[elbow_index].x, landmarks[elbow_index].y])
                wrist = np.array([landmarks[wrist_index].x, landmarks[wrist_index].y])
                shoulder_elbow_wrist_angle = calculate_angle(shoulder, elbow, wrist)
                cv2.putText(
                    image,
                    f"Angle: {shoulder_elbow_wrist_angle:.2f}",
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 255, 0),
                    2,
                )

                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                )

            cv2.imshow("Pose Estimation", image)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

    cap.release()
    cv2.destroyAllWindows()

In [2]:
cap = cv2.VideoCapture("punches_videos/jab/jab1.mp4")
get_pose_landmarks(cap)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


AttributeError: 'NoneType' object has no attribute 'landmark'